In [2]:
:dep graph-ds = { path = ".", default-features = false }
:dep anyhow = "1.0"
// :dep h3o = "0.3.0"
// :dep plotters = "0.3.4"

In [3]:
use graph_ds::{Graph, Node, Edge, AStarResult};
use graph_ds::hexagon_graph::{cell_graph_from_mpk, h3_network_from_osm, h3_network_from_gtfs, cell::Cell, h3cell::H3Cell, osm::OSMLayer};

use std::time::Instant;
use std::sync::{Arc, RwLock, RwLockWriteGuard};
use std::collections::BTreeSet;
use std::collections::hash_map::DefaultHasher;
use std::hash::{Hash, Hasher};


In [4]:
let set_1 = BTreeSet::from([1, 2, 3]);
let set_2 = BTreeSet::from([3, 2, 1]);

let mut h1 = DefaultHasher::new();
set_1.hash(&mut h1);
let mut h2 = DefaultHasher::new();
set_2.hash(&mut h2);

assert!(h1.finish() == h2.finish());

## Cell Graph

In [14]:
let mut graph : Graph<Cell> = cell_graph_from_mpk("../resources/de_mirage_hexagons.mpk")?;
println!("Graph has {} nodes", graph.nr_nodes());

Graph has 7935 nodes


In [15]:
graph.node_hash()

7302240096543813149

In [7]:
fn heuristic(start_cell: &Cell, end_cell: &Cell) -> f64 {
    let dx = (start_cell.a - end_cell.a).abs();
    let dy = (start_cell.b - end_cell.b).abs();
    let dz = (start_cell.a + start_cell.b - end_cell.a - end_cell.b).abs();
    let dlayer = (start_cell.layer - end_cell.layer).abs();
    ((dx + dy + dz) as f64 / 2.0 + dlayer as f64) * (start_cell.radius * 2) as f64
}

In [8]:
let start = graph.get_random_node().unwrap();
let end = graph.get_random_node().unwrap();

println!("Start: {:?}", start);
println!("End: {:?}", end);

Start: Cell { a: -25, b: 27, radius: 24, layer: 0 }
End: Cell { a: -42, b: -5, radius: 24, layer: 0 }


In [11]:
println!("BFS start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let (bfs_path, bfs_distance) = graph.bfs(start, Some(end), &None)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", bfs_distance[0].unwrap());
println!("path length: {:?}", bfs_path.as_ref().unwrap().len());

println!("---");

// println!("matrix bfs");
// let now = Instant::now();
// graph.matrix_bfs_distance(vec![start; 10], Some(vec![end; 10]), true);
// println!("time: {:?} µs", now.elapsed().as_micros());
// let distances = distances.into_iter().flatten().collect::<Vec<_>>();
// println!("distances: {:?}", distances);

println!("---");

println!("AStar start: {:?}, end: {:?}", start, end);
let now = Instant::now();
let astar_result = graph.astar(start, Some(end), &None, None, heuristic)?;
println!("time: {:?} µs", now.elapsed().as_micros());
println!("distance: {:?}", astar_result.distances.first().unwrap());
println!("path length: {:?}", astar_result.path.unwrap().len());

BFS start: Cell { a: -25, b: 27, radius: 24, layer: 0 }, end: Cell { a: -42, b: -5, radius: 24, layer: 0 }
[backtrace] found start node
time: 582 µs
distance: 106.0
path length: 60
---
---
AStar start: Cell { a: -25, b: 27, radius: 24, layer: 0 }, end: Cell { a: -42, b: -5, radius: 24, layer: 0 }
[backtrace] found start node
time: 929 µs
distance: 2784.315170288086
path length: 61


## OSM + GTFS H3 Graph

In [12]:
let mut osm_g : Graph<H3Cell> = h3_network_from_osm("../resources/copenhagen-processed.osm.pbf", OSMLayer::Walking)?;
println!("Graph has {} nodes, hash: {}", osm_g.nr_nodes(), osm_g.node_hash());

processing osm pbf file: ../resources/copenhagen-processed.osm.pbf
converted OSM file into 446019 edges
Graph has 364511 nodes, hash: 1372348476429501945


In [13]:
let mut gtfs_g : Graph<H3Cell> = h3_network_from_gtfs("../resources/rejseplanen.zip")?;
println!("Graph has {} nodes, hash: {}", gtfs_g.nr_nodes(), gtfs_g.node_hash());

getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
Graph has 103525 nodes, hash: 11658248065950047293
